# Segmented Representations

One common representation in evolutionary algorithms (EA) is that of a "segmented representation."  That is, each individual is comprised of a sequence of segments, which are themselves fixed-length sequences, and are usually binary, but needn't be.  Each segment represents a salient feature, such as a rule in a Pitt Approach system, or a convolutional layer and its hyperparameters, as is the case for Multi-node Evolutionary Neural Networks for Deep Learning (MENNDL).

There are two broad categories for these systems: those that have a fixed number of such segments, as is the case currently for MENNDL, and a dynamic number of segments, as is the case for Pitt Approach classifiers.

In this notebook we look at LEAP support for segmented representations, starting with initializers and decoders, and then looking at the mutation pipeline operator.  We then plug all that into a simple EA example.


In [1]:
import sys
import random
import functools
from pprint import pprint, pformat
import numpy as np
from toolz import pipe

from leap_ec.individual import Individual
from leap_ec.ops import pool, cyclic_selection, clone

from leap_ec.segmented_rep.initializers import create_segmented_sequence
from leap_ec.segmented_rep.decoders import SegmentedDecoder
from leap_ec.segmented_rep.ops import apply_mutation, add_segment, remove_segment, copy_segment

from leap_ec.binary_rep.initializers import create_binary_sequence
from leap_ec.binary_rep.ops import genome_mutate_bitflip
from leap_ec.binary_rep.decoders import BinaryToIntDecoder

from leap_ec.real_rep.initializers import create_real_vector
from leap_ec.real_rep.ops import genome_mutate_gaussian

## Binary genomes

We first look at segmented representations with segments that use a binary representaion.

In [2]:
# Create a genome of four segments of five binary digits.
seg = create_segmented_sequence(4, create_binary_sequence(5))
print(seg)

[[0, 0, 1, 0, 0], [1, 1, 0, 0, 1], [1, 0, 0, 1, 1], [0, 0, 1, 0, 0]]


In [3]:
# Now create five genomes of varying length by passing in a function for `length` that provides an
# integer drawn from a distribution.
seqs = [] # Save sequences for next step
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=5), create_binary_sequence(5))
    print(i, seq)
    seqs.append(seq)

0 [[1, 1, 1, 0, 1], [0, 1, 1, 0, 1], [1, 0, 1, 1, 1], [0, 1, 1, 1, 0]]
1 [[1, 1, 1, 0, 0], [1, 0, 0, 1, 1], [1, 1, 0, 1, 0], [0, 0, 1, 1, 0]]
2 [[1, 1, 0, 1, 1], [1, 0, 0, 1, 0], [0, 1, 1, 0, 1]]
3 [[0, 1, 0, 0, 0], [0, 0, 1, 0, 1]]
4 [[0, 0, 0, 0, 1], [1, 0, 0, 0, 0], [0, 0, 1, 1, 1], [0, 1, 0, 1, 0], [1, 1, 1, 1, 0]]


Now let's see about decoding those segments.  The segmented representation relies on a secondary decoder that's applied to each segment.  In this case, we'll just use a simple binary to int decoder on the segments we created in the previous step.

In [4]:
# We want each segment to have two integers from the five bits.
decoder = SegmentedDecoder(BinaryToIntDecoder(2,3)) 

for i, seq in enumerate(seqs):
    vals = decoder.decode(seq)
    print(i, vals)

0 [[3, 5], [1, 5], [2, 7], [1, 6]]
1 [[3, 4], [2, 3], [3, 2], [0, 6]]
2 [[3, 3], [2, 2], [1, 5]]
3 [[1, 0], [0, 5]]
4 [[0, 1], [2, 0], [0, 7], [1, 2], [3, 6]]


In [5]:
# And now for mutation, which shows that, on average, a single value is changed in an example individual.  The
# takeaway here is that segmented mutation just uses a mutator from another representation and naively applies it.

original = Individual([[0,0],[1,1]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_bitflip))
print('mutated:', mutated)

original: [[0, 0], [1, 1]]
mutated: [[0, 0], [1, 1]]


## Real-valued genomes

Now we demonstrate the same process using a real-valued representation.

In [6]:
# Create five segmented sequences that vary from 1 to 3 segments
bounds = ((-5.12,5.12), (-1,1), (-10,10)) # three reals and their respective bounds for sampling
seqs = []
for i in range(5):
    seq = create_segmented_sequence(functools.partial(random.randint, a=1,b=3), 
                                    create_real_vector(bounds))
    seqs.append(seq)

# Just for fun, now add a genome that has exactly 5 segments
seqs.append(create_segmented_sequence(5, create_real_vector(bounds)))

for i, s in enumerate(seqs):
    print(i, pformat(s, indent=2))

0 [[-0.5176825288591136, 0.5327185262219665, -7.5516764980186535]]
1 [ [4.20964710890548, 0.7910191070814543, 1.8802851802399871],
  [-1.30638615990487, -0.881181549246117, -9.07248199856199]]
2 [ [-2.8312527518236448, -0.7945028211915537, -6.888479756422637],
  [-1.4730767089654924, 0.6855275531876028, 8.166271152032433]]
3 [ [-1.3564074677604663, -0.9578454252035502, -2.0229357050554437],
  [-0.6169318979288754, 0.9111811397046765, 8.7235664541811]]
4 [ [-0.8633020880460887, -0.27630797019127873, -2.8971459492907297],
  [-4.228045273555841, -0.09374668183641077, 9.072038954685024],
  [-2.853171987223876, 0.8313550032008383, 9.90947320574718]]
5 [ [-2.015956196662512, -0.027627896008496355, -4.043638978693269],
  [2.8625467441507295, 0.5509958922043023, -4.33878788662531],
  [4.249217803716092, 0.5088249430883702, -1.6174217746560338],
  [4.621121971792143, -0.21613385519258022, 4.112426903679939],
  [-3.1309030625095295, -0.02835525175511977, 4.2321403287228705]]


Now we repeat the application of the segmented mutation operator, but this time to real-valued genomes.

In [7]:
original = Individual([[0.0,0.0],[1.0,1.0],[-1.0,0.0]])
print('original:', original)
mutated = next(apply_mutation(iter([original]),mutator=genome_mutate_gaussian(std=1.0)))
print('mutated:', mutated)

original: [[0.0, 0.0], [1.0, 1.0], [-1.0, 0.0]]
mutated: [[0.0, 2.3774821131746577], [1.0, 1.0], [-1.663943746350972, 0.0]]


# Other pipeline operators

Besides the aformentioned `apply_mutation`, segmented representations have other pipeline operators, which are:

* `add_segment()`, possibly add a new segment
* `remove_segment()`, possibly remove a segment
* `copy_segment()`, possibly select and copy an existing segment


In [3]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               remove_segment(probability=1.0), 
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop))

pop: [Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None),
 Individual([[0, 0], [1, 1]], None, None)]
new_pop: [Individual([[0, 0]], None, None),
 Individual([[1, 1]], None, None),
 Individual([[0, 0]], None, None),
 Individual([[1, 1]], None, None),
 Individual([[1, 1]], None, None)]


In [5]:
# demonstrate pipe by running existing sequence through a number of operators
pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))
new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               copy_segment(probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None)]
new_pop: [        Individual([[0, 0], [0, 0], [1, 1]], None, None),
         Individual([[1, 1], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [1, 1]], None, None),
         Individual([[0, 0], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [0, 0], [1, 1]], None, None)]


In [7]:
# lastly, demonstrate add_segment, which generates an entirely new segment
test_sequence = [12345]  # just an arbitrary sequence for testing

def gen_sequence():
    """ return an arbitrary static test_sequence """
    return test_sequence

pop = [Individual([[0,0],[1,1]]) for x in range(5)]
print('pop:', pformat(pop, indent=5))

new_pop = pipe(pop, 
               cyclic_selection,
               clone,
               add_segment(seq_initializer=gen_sequence, probability=1.0),
               pool(size=len(pop)))
print('new_pop:', pformat(new_pop, indent=9))

pop: [    Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None),
     Individual([[0, 0], [1, 1]], None, None)]
new_pop: [        Individual([[0, 0], [12345], [1, 1]], None, None),
         Individual([[0, 0], [12345], [1, 1]], None, None),
         Individual([[12345], [0, 0], [1, 1]], None, None),
         Individual([[0, 0], [1, 1], [12345]], None, None),
         Individual([[0, 0], [1, 1], [12345]], None, None)]
